<a href="https://colab.research.google.com/github/deepakri201/MIDRC_colab/blob/main/MIDRC_add_DICOM_to_datastore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

One dicom datastore (deepa) will have raw, SEG and SR (needed for OHIF viewer). Another dicom datastore (deepa_seg_and_sr) will only have SEG and SR. This is the one that will be used to create the DICOM metadata table for plotting etc (bwh-midrc-rapid-res-1655321320.midrc_dicom_us.midrc_with_seg_sr_dk). 

Having issues with dicom datastore with the raw CT. 
- Where is the original data in the bucket located? 
- Tried exporting from the known dicom datastore (dataset midrc, datastore midrc-dicom) to my own bucket, but permission denied. Thought I could do that and ingest into my own DICOM datastore. If I didn't find the name of the original. 

Notes: 

Do this for one subject: For now, use `dcmweb retrieve` to get raw data from DICOM datastore to local. Then use `dcmweb store` to store raw data  in the new DICOM datastore (called deepa). Then import SEG from bucket to DICOM datastore. OHIF URL though is not working yet... 

Instead of doing it this way - export from DICOM datastore to bucket myself (midrc-dicom) and reingest into my own DICOM datastore -- if I don't find out the name of the bucket from A/N. It will take a while to do this each time though... 


# Parameterization

In [1]:
!gcloud config set project  bwh-midrc-rapid-res-1655321320

Updated property [core/project].


In [2]:
from google.colab import auth
auth.authenticate_user()

In [21]:
project_name = "bwh-midrc-rapid-res-1655321320"
# bucket_name = "midrc-analysis-bwh"
bucket_name = "midrc-analysis-bwh-dk"
bucket_path = "bpr-results"
bucket_path_sr = "structured_reports/" 
bucket_path_sr_and_seg = "seg_and_sr"

dataset_table_id = "midrc_dicom_us" # This already exists 
table_view_id_name = "ct_limited_open_a1_r1_dk"
# table_id = "bwh-midrc-rapid-res-1655321320.midrc_dicom_us.ct_limited_open_a1_r1_dk"

# dataset_id = "midrc" # already created
# dicom_store_id = "midrc-dicom"
dataset_id = "midrc"
dicom_store_id = "midrc-dicom"
location_id = "us-central1"

my_dataset_id = "deepa"
my_dicom_store_id = "deepa"
my_dicom_store_id_seg_and_sr = "deepa_seg_and_sr"
# my_dicom_store_id2 = "deepa2"
# raw_data_bucket = 'gs://midrc-analysis-bwh-dk/raw_data'
raw_data_bucket = 'midrc-analysis-bwh-dk/raw_data'

bucket_name_midrc_data = "midrc-data-bwh"


# Environment setup

**Install BodyPartRegression package**

In [4]:
#%%capture 
import shutil
import os
if os.path.isdir('/content/BodyPartRegression'):
  shutil.rmtree('/content/BodyPartRegression')
!pip install torch==1.8.1 pytorch-lightning==1.2.10 torchtext==0.9.1 torchvision==0.9.1 torchaudio==0.8.1 dataclasses==0.6
!pip install bpreg
!git clone https://github.com/MIC-DKFZ/BodyPartRegression.git
# !pip install torch==1.8.1 pytorch-lightning==1.2.10 torchtext==0.9.1
!pip install SimpleITK
!pip install pydicom

import bpreg 
import seaborn as sb 
import pandas as pd 
import SimpleITK as sitk
import glob
import matplotlib.pyplot as plt 

!pip install opencv-python-headless==4.1.2.30 # https://stackoverflow.com/questions/70537488/cannot-import-name-registermattype-from-cv2-cv2/70547274
from BodyPartRegression.docs.notebooks.utils import * 

print (pd.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.1/804.1 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.9/841.9 KB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.9/176.9 KB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 KB 49.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492036 sha256=7ca98b57da840fc0d7ff2075a4fd3cde38aeee0d3d85d28605dff0588ffea926
  Stored in directory: /root/.cache/pip/wheels/a0/0b/ee/e6994fadb42c1354dcccb139b0bf2795271bddfe6253ccdf11
Successfully built future
  Attempting

In [5]:
from google.cloud import bigquery

In [6]:
!pip install https://github.com/GoogleCloudPlatform/healthcare-api-dicomweb-cli/archive/v1.0.2.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     - 50.4 kB 2.4 MB/s 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 KB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for dcmweb: filename=dcmweb-1.0.2-py3-none-any.whl size=26439 sha256=721da8db1858257e5fcc46b4c83a2031556150ac6424d59cac66483ca3957a05
  Stored in directory: /tmp/pip-ephem-wheel-cache-1v4tqnw_/wheels/72/cb/c0/8e1b6fd05e3af6d751c98b4a316bbe3c5658baedd3f5af98ad
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116949 sha256=b7172dc7909583a958b88996c1c80a5fe04e31a975c7b41e7f3d68df90704db2
  Stored in directory: /root/.cache/pip/wheels/5b/eb/43/7295e71293b218ddfd627f935229bf54af9018add7fbb5aac6
  Created wheel for hurry.filesize: filename=hurry.filesize-0.9-py3-none-any.whl size=4134 

In [7]:
# install dcm2niix
!sudo apt-get install dcm2niix 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libyaml-cpp0.6
Suggested packages:
  pigz
The following NEW packages will be installed:
  dcm2niix libyaml-cpp0.6
0 upgraded, 2 newly installed, 0 to remove and 23 not upgraded.
Need to get 300 kB of archives.
After this operation, 1,110 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libyaml-cpp0.6 amd64 0.6.2-4ubuntu1 [124 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 dcm2niix amd64 1.0.20181125-1build1 [176 kB]
Fetched 300 kB in 1s (386 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend 

In [8]:
#install nrrd
!pip install pynrrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
#Install Plastimatch

!sudo apt install plastimatch 
!echo $(plastimatch --version)

if os.path.isdir('/content/pyplastimatch'):
  try:
    shutil.rmtree('/content/pyplastimatch')
  except OSError as err:
    print("Error: %s : %s" % ("pyplastimatch", err.strerror)) 
# !git clone https://github.com/denbonte/pyplastimatch/ pyplastimatch
!git clone https://github.com/AIM-Harvard/pyplastimatch.git 

# from pyplastimatch import pyplastimatch as pypla
from pyplastimatch.pyplastimatch import pyplastimatch as pypla

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libdcmtk14 libdlib-data libdlib19 libfftw3-single3 libinsighttoolkit4.13
  libminc2-5.2.0 libnifti2
Suggested packages:
  libfftw3-bin libfftw3-dev
The following NEW packages will be installed:
  libdcmtk14 libdlib-data libdlib19 libfftw3-single3 libinsighttoolkit4.13
  libminc2-5.2.0 libnifti2 plastimatch
0 upgraded, 8 newly installed, 0 to remove and 23 not upgraded.
Need to get 80.1 MB of archives.
After this operation, 169 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libdcmtk14 amd64 3.6.4-2.1build2 [4,682 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 libdlib-data all 19.10-3build1 [63.4 MB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 libdlib19 amd64 19.10-3build1 [3,773 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/main amd64 libfftw3-singl

In [10]:
import glob
import numpy as np
from bpreg.scripts.bpreg_inference import bpreg_inference
from google.cloud import storage
import nrrd
import json
import matplotlib.pyplot as plt
from matplotlib import transforms
from matplotlib.colors import ListedColormap

# Queries

In [11]:
# %%bigquery ct_limited_open_a1_r1 --project=bwh-midrc-rapid-res-1655321320 

# WITH
#   nlst_instances_per_series AS (
#     SELECT
#       DISTINCT(StudyInstanceUID),
#       SeriesInstanceUID,
#       COUNT(DISTINCT(SOPInstanceUID)) AS num_instances,
#       COUNT(DISTINCT(ARRAY_TO_STRING(ImagePositionPatient,"/"))) AS position_count,
#       COUNT(DISTINCT(ARRAY_TO_STRING(ImageOrientationPatient,"/"))) AS orientation_count,
#       MIN(SAFE_CAST(SliceThickness AS float64)) AS min_SliceThickness,
#       MAX(SAFE_CAST(SliceThickness AS float64)) AS max_SliceThickness,
#       MIN(SAFE_CAST(ImagePositionPatient[SAFE_OFFSET(2)] AS float64)) as min_SliceLocation, 
#       MAX(SAFE_CAST(ImagePositionPatient[SAFE_OFFSET(2)] AS float64)) as max_SliceLocation,
#       STRING_AGG(DISTINCT(SAFE_CAST("LOCALIZER" IN UNNEST(ImageType) AS string)),"") AS has_localizer
#     FROM
#       bwh-midrc-rapid-res-1655321320.midrc_dicom_us.dicom_all
#     WHERE
#       (collection_id = "Open-R1" or collection_id = "Open-A1") and Modality = "CT"
#     GROUP BY
#       StudyInstanceUID,
#       SeriesInstanceUID
#       ), 
#   nlst_values_per_series AS (
#     SELECT 
#     ANY_VALUE(dicom_all.PatientID) AS PatientID,
#     dicom_all.SeriesInstanceUID,
#     ANY_VALUE(nlst_instances_per_series.num_instances) AS num_instances,
#     ANY_VALUE(nlst_instances_per_series.max_SliceThickness) AS SliceThickness,
#     ANY_VALUE((nlst_instances_per_series.max_SliceLocation - nlst_instances_per_series.min_SliceLocation)) AS PatientHeightScanned
#   FROM
#     bwh-midrc-rapid-res-1655321320.midrc_dicom_us.dicom_all AS dicom_all
#   JOIN
#     nlst_instances_per_series
#   ON
#     dicom_all.SeriesInstanceUID = nlst_instances_per_series.SeriesInstanceUID
#   WHERE
#     min_SliceThickness >= 1.5 
#     AND max_SliceThickness <= 3.5 
#     AND nlst_instances_per_series.num_instances > 100
#     AND nlst_instances_per_series.num_instances/nlst_instances_per_series.position_count = 1
#     AND nlst_instances_per_series.orientation_count = 1
#     AND has_localizer = "false"
#   GROUP BY
#     SeriesInstanceUID
#   )
#   SELECT 
#     dicom_all.PatientID,
#     dicom_all.StudyInstanceUID,
#     dicom_all.SeriesInstanceUID,
#     dicom_all.SOPInstanceUID,
#     dicom_all.collection_id,
#     dicom_all.PatientAge,
#     dicom_all.PatientWeight,
#     nlst_values_per_series.num_instances,
#     nlst_values_per_series.SliceThickness,
#     nlst_values_per_series.PatientHeightScanned
#   FROM
#     bwh-midrc-rapid-res-1655321320.midrc_dicom_us.dicom_all AS dicom_all
#   JOIN
#     nlst_values_per_series 
#   ON
#     dicom_all.SeriesInstanceUID = nlst_values_per_series.SeriesInstanceUID

In [12]:
# print(len(set(ct_limited_open_a1_r1['PatientID'].values)))
# print(len(set(ct_limited_open_a1_r1['StudyInstanceUID'].values)))
# print(len(set(ct_limited_open_a1_r1['SeriesInstanceUID'].values)))

In [13]:
# %%bigquery ct_limited_open_a1_r1 --project=bwh-midrc-rapid-res-1655321320 

query = """
WITH
  nlst_instances_per_series AS (
    SELECT
      # DISTINCT(StudyInstanceUID),
      StudyInstanceUID,
      SeriesInstanceUID,
      COUNT(DISTINCT(SOPInstanceUID)) AS num_instances,
      COUNT(DISTINCT(ARRAY_TO_STRING(ImagePositionPatient,"/"))) AS position_count,
      COUNT(DISTINCT(ARRAY_TO_STRING(PixelSpacing,"/"))) AS pixel_spacing_count,
      COUNT(DISTINCT(ARRAY_TO_STRING(ImageOrientationPatient,"/"))) AS orientation_count,
      MIN(SAFE_CAST(SliceThickness AS float64)) AS min_SliceThickness,
      MAX(SAFE_CAST(SliceThickness AS float64)) AS max_SliceThickness,
      MIN(SAFE_CAST(ImagePositionPatient[SAFE_OFFSET(2)] AS float64)) as min_SliceLocation, 
      MAX(SAFE_CAST(ImagePositionPatient[SAFE_OFFSET(2)] AS float64)) as max_SliceLocation,
      STRING_AGG(DISTINCT(SAFE_CAST("LOCALIZER" IN UNNEST(ImageType) AS string)),"") AS has_localizer,
      ANY_VALUE(dicom_all.ImageOrientationPatient) AS ImageOrientationPatient
    FROM
      bwh-midrc-rapid-res-1655321320.midrc_dicom_us.dicom_all
    WHERE
      (collection_id = "Open-R1" or collection_id = "Open-A1") and Modality = "CT"
    GROUP BY
      StudyInstanceUID,
      SeriesInstanceUID
      ), 

    distinct_slice_location_difference_values AS (
  SELECT 
      # DISTINCT(SAFE_CAST(ImagePositionPatient[SAFE_OFFSET(2)] AS NUMERIC) - LAG(SAFE_CAST(ImagePositionPatient[SAFE_OFFSET(2)] AS NUMERIC),1) OVER(partition by SeriesInstanceUID ORDER BY SAFE_CAST(ImagePositionPatient[SAFE_OFFSET(2)] AS NUMERIC) DESC)) AS SliceLocation_difference,
      DISTINCT(TRUNC(SAFE_CAST(ImagePositionPatient[SAFE_OFFSET(2)] AS NUMERIC),1) - LAG(TRUNC(SAFE_CAST(ImagePositionPatient[SAFE_OFFSET(2)] AS NUMERIC),1),1) OVER(partition by SeriesInstanceUID ORDER BY TRUNC(SAFE_CAST(ImagePositionPatient[SAFE_OFFSET(2)] AS NUMERIC),1) DESC)) AS SliceLocation_difference,
      SeriesInstanceUID,
      StudyInstanceUID
  FROM
      # `bigquery-public-data.idc_current.dicom_all`
      bwh-midrc-rapid-res-1655321320.midrc_dicom_us.dicom_all

  ),

  nlst_values_per_series AS (
  SELECT 
    # ANY_VALUE(dicom_all.PatientID) AS PatientID,
    # dicom_all.SeriesInstanceUID,
    distinct_slice_location_difference_values.SeriesInstanceUID AS SeriesInstanceUID, 
    ANY_VALUE(nlst_instances_per_series.num_instances) AS num_instances,
    ANY_VALUE(nlst_instances_per_series.max_SliceThickness) AS SliceThickness,
    ANY_VALUE((nlst_instances_per_series.max_SliceLocation - nlst_instances_per_series.min_SliceLocation)) AS PatientHeightScanned, 
    COUNT(distinct_slice_location_difference_values.SliceLocation_difference) as num_differences,
    MAX(ABS(distinct_slice_location_difference_values.SliceLocation_difference)) as max_difference,
    MIN(ABS(distinct_slice_location_difference_values.SliceLocation_difference)) as min_difference,
    ANY_VALUE(nlst_instances_per_series.ImageOrientationPatient) AS ImageOrientationPatient
  FROM
    # bwh-midrc-rapid-res-1655321320.midrc_dicom_us.dicom_all AS dicom_all
    nlst_instances_per_series
  JOIN
    # nlst_instances_per_series
    distinct_slice_location_difference_values
  ON
    # dicom_all.SeriesInstanceUID = nlst_instances_per_series.SeriesInstanceUID
    nlst_instances_per_series.SeriesInstanceUID = distinct_slice_location_difference_values.SeriesInstanceUID 
  WHERE
    nlst_instances_per_series.min_SliceThickness >= 1.5
    AND nlst_instances_per_series.max_SliceThickness <= 3.5
    AND nlst_instances_per_series.num_instances > 100
    AND nlst_instances_per_series.num_instances/nlst_instances_per_series.position_count = 1
    AND nlst_instances_per_series.pixel_spacing_count = 1
    AND nlst_instances_per_series.orientation_count = 1
    AND has_localizer = "false" 
    AND ABS(SAFE_CAST(nlst_instances_per_series.ImageOrientationPatient[SAFE_OFFSET(0)] AS float64)) > ABS(SAFE_CAST(nlst_instances_per_series.ImageOrientationPatient[SAFE_OFFSET(1)] AS float64))
    AND ABS(SAFE_CAST(nlst_instances_per_series.ImageOrientationPatient[SAFE_OFFSET(0)] AS float64)) > ABS(SAFE_CAST(nlst_instances_per_series.ImageOrientationPatient[SAFE_OFFSET(2)] AS float64))
    AND ABS(SAFE_CAST(nlst_instances_per_series.ImageOrientationPatient[SAFE_OFFSET(4)] AS float64)) > ABS(SAFE_CAST(nlst_instances_per_series.ImageOrientationPatient[SAFE_OFFSET(3)] AS float64))
    AND ABS(SAFE_CAST(nlst_instances_per_series.ImageOrientationPatient[SAFE_OFFSET(4)] AS float64)) > ABS(SAFE_CAST(nlst_instances_per_series.ImageOrientationPatient[SAFE_OFFSET(5)] AS float64))
  GROUP BY
    distinct_slice_location_difference_values.SeriesInstanceUID

  )

  SELECT 
    dicom_all.PatientID,
    dicom_all.StudyInstanceUID,
    dicom_all.SeriesInstanceUID,
    dicom_all.SOPInstanceUID, 
    dicom_all.collection_id, 
    dicom_all.PatientAge,
    dicom_all.PatientWeight,
    nlst_values_per_series.num_instances,
    nlst_values_per_series.SliceThickness, 
    nlst_values_per_series.PatientHeightScanned,
    nlst_values_per_series.num_differences,
    nlst_values_per_series.max_difference, 
    nlst_values_per_series.min_difference
  FROM 
    bwh-midrc-rapid-res-1655321320.midrc_dicom_us.dicom_all AS dicom_all
  JOIN 
    nlst_values_per_series
  ON 
    dicom_all.SeriesInstanceUID = nlst_values_per_series.SeriesInstanceUID 
  WHERE
    nlst_values_per_series.num_differences <= 2
    AND nlst_values_per_series.max_difference/nlst_values_per_series.min_difference < 2
"""

client = bigquery.Client(project=project_name)

table_id = '.'.join([project_name, dataset_table_id, table_view_id_name])
job_config = bigquery.QueryJobConfig(destination=table_id)

# Try to create table 
try: 
  print ('trying to create table: ' + str(table_id))
  query_job = client.query(query, job_config=job_config) 
  result = query_job.result()  
  print ('created table: ' + str(table_id))
# If table already exists, don't do anything
except: 
  print('table ' + str(table_id) + ' already exists')
  pass 



trying to create table: bwh-midrc-rapid-res-1655321320.midrc_dicom_us.ct_limited_open_a1_r1_dk
table bwh-midrc-rapid-res-1655321320.midrc_dicom_us.ct_limited_open_a1_r1_dk already exists


In [14]:
# Get the dataframe from the table 

client = bigquery.Client(project=project_name)
table_id = '.'.join([project_name, dataset_table_id, table_view_id_name])

query_view = f"""
  SELECT 
    * 
  FROM 
    {table_id}
  """ 
job_config = bigquery.QueryJobConfig()
result = client.query(query_view, job_config=job_config) 
ct_limited_open_a1_r1 = result.to_dataframe()


In [15]:
ct_limited_open_a1_r1

,PatientID,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,collection_id,PatientAge,PatientWeight,num_instances,SliceThickness,PatientHeightScanned,num_differences,max_difference,min_difference
0,10000364-1310150,2.16.840.1.114274.1818.56277717926260378856921...,2.16.840.1.114274.1818.46652472987550009305366...,2.16.840.1.114274.1818.46611042193109500771675...,Open-A1,062Y,94.348,152,1.5,226.5,1,1.500000000,1.500000000
1,10000364-1310150,2.16.840.1.114274.1818.56277717926260378856921...,2.16.840.1.114274.1818.46652472987550009305366...,2.16.840.1.114274.1818.56958767487481437501222...,Open-A1,062Y,94.348,152,1.5,226.5,1,1.500000000,1.500000000
2,10000364-1310150,2.16.840.1.114274.1818.56277717926260378856921...,2.16.840.1.114274.1818.46652472987550009305366...,2.16.840.1.114274.1818.51535999809052778591562...,Open-A1,062Y,94.348,152,1.5,226.5,1,1.500000000,1.500000000
3,10000364-1310150,2.16.840.1.114274.1818.56277717926260378856921...,2.16.840.1.114274.1818.46652472987550009305366...,2.16.840.1.114274.1818.53325924708374518603071...,Open-A1,062Y,94.348,152,1.5,226.5,1,1.500000000,1.500000000
4,10000364-1310150,2.16.840.1.114274.1818.56277717926260378856921...,2.16.840.1.114274.1818.46652472987550009305366...,2.16.840.1.114274.1818.53878510159989617247990...,Open-A1,062Y,94.348,152,1.5,226.5,1,1.500000000,1.500000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75550,10000364-6509816,1.3.6.1.4.1.5962.99.1.3492129840.282517718.161...,1.3.6.1.4.1.5962.99.1.3492129840.282517718.161...,1.3.6.1.4.1.5962.99.1.3492129840.282517718.161...,Open-A1,None,None,229,1.5,342.0,1,1.500000000,1.500000000
75551,10000364-6509816,1.3.6.1.4.1.5962.99.1.3492129840.282517718.161...,1.3.6.1.4.1.5962.99.1.3492129840.282517718.161...,1.3.6.1.4.1.5962.99.1.3492129840.282517718.161...,Open-A1,None,None,229,1.5,342.0,1,1.500000000,1.500000000
75552,10000364-6509816,1.3.6.1.4.1.5962.99.1.3492129840.282517718.161...,1.3.6.1.4.1.5962.99.1.3492129840.282517718.161...,1.3.6.1.4.1.5962.99.1.3492129840.282517718.161...,Open-A1,None,None,229,1.5,342.0,1,1.500000000,1.500000000
75553,10000364-6509816,1.3.6.1.4.1.5962.99.1.3492129840.282517718.161...,1.3.6.1.4.1.5962.99.1.3492129840.282517718.161...,1.3.6.1.4.1.5962.99.1.3492129840.282517718.161...,Open-A1,None,None,229,1.5,342.0,1,1.500000000,1.500000000


In [16]:
print(len(set(ct_limited_open_a1_r1['PatientID'].values)))
print(len(set(ct_limited_open_a1_r1['StudyInstanceUID'].values)))
print(len(set(ct_limited_open_a1_r1['SeriesInstanceUID'].values)))


188
199
389


# Create DICOM dataset, datastore

In [17]:
def create_gcp_healthcare_dataset(project_name, location_id, dataset_id):
  """Creates a GCP healthcare dataset using gcloud commands if it doesn't exist

  Inputs: 
    project_name : the project id 
    location_id  : the location of the project, e.g. us-central1
    dataset_id   : the name of the dataset to create 

  Outputs:
    creates the dataset if it doesn't exist 

  """
  
  # First let's list the datasets that we already have for our particular project_id and location_id
  datasets = !gcloud healthcare datasets list --project $project_name --location $location_id --format="value(ID)" 
  print ('datasets that exist for project_id ' + str(project_name) + ', location ' + str(location_id) + ': ' + str(datasets))
  
  # If the dataset doesn't exist, create it 
  if not (dataset_id in datasets):
    try:
      !gcloud healthcare datasets create --project $project_name $dataset_id --location $location_id
      print ("Created dataset " + str(dataset_id))
    except OSError as err:
      print("Error: %s : %s" % ("Unable to create dataset", err.strerror)) 

  # As a check, we'll list the datasets again.
  datasets = !gcloud healthcare datasets list --project $project_name --location $location_id --format="value(ID)" 
  print ('datasets that exist for project_id ' + str(project_name) + ', location ' + str(location_id) + ': ' + str(datasets))

  return

In [18]:
def create_gcp_healthcare_dicom_datastore(project_name, location_id, dataset_id, datastore_id):
  """Creates a GCP healthcare DICOM datastore using gcloud commands if it doesn't exist

  Inputs: 
    project_name : the project id 
    location_id  : the location of the project, e.g. us-central1
    dataset_id   : the dataset id
    datastore_id : the DICOM datastore to create 

  Outputs:
    creates the DICOM datastore if it doesn't exist

  """

  # First list the datastores that exist in the dataset
  datastores = !gcloud healthcare dicom-stores list --project $project_name --location $location_id --dataset $dataset_id --format="value(ID)"
  print ('datastores that exist for project_id ' + str(project_name) + ', location ' + str(location_id) + ', dataset ' + str(dataset_id) + ': ' + str(datastores))

  # If the dicom_store_id doesn't exist, create it
  if not (datastore_id in datastores):
    try:
      !gcloud healthcare dicom-stores create $datastore_id --project $project_name --location $location_id --dataset $dataset_id --format="value(ID)" 
      print ("Created DICOM datastore " + str(datastore_id))
    except OSError as err:
      print("Error: %s : %s" % ("Unable to create datastore", err.strerror)) 

  # As a check, we'll list the datastores again.
  datastores = !gcloud healthcare dicom-stores list --project $project_name --location $location_id --dataset $dataset_id --format="value(ID)"
  print ('datastores that exist for project_id ' + str(project_name) + ', location ' + str(location_id) + ', dataset ' + str(dataset_id) + ': ' + str(datastores))

  return

Create DICOM datastores for both the one that will hold raw/SEG/SR and one for just SEG/SR

In [30]:
# Delete DICOM datastore if it already exists 

!gcloud healthcare dicom-stores delete $my_dicom_store_id --project $project_name --dataset $my_dataset_id --format="value(ID)" 

You are about to delete dicomStore [deepa]

Do you want to continue (Y/n)?  Y

Deleted dicomStore [deepa].



In [42]:
# Delete DICOM datastore if it already exists 

!gcloud healthcare dicom-stores delete $my_dicom_store_id_seg_and_sr --project $project_name --dataset $my_dataset_id --format="value(ID)" 

You are about to delete dicomStore [deepa_seg_and_sr]

Do you want to continue (Y/n)?  Y

Deleted dicomStore [deepa_seg_and_sr].



In [37]:
create_gcp_healthcare_dataset(project_name, location_id, my_dataset_id)


datasets that exist for project_id bwh-midrc-rapid-res-1655321320, location us-central1: ['afedorov-hc-dataset', 'midrc', 'midrc-dk']
Create request issued for: [deepa]
Created dataset [deepa].
Created dataset deepa
datasets that exist for project_id bwh-midrc-rapid-res-1655321320, location us-central1: ['afedorov-hc-dataset', 'deepa', 'midrc', 'midrc-dk']


In [31]:
create_gcp_healthcare_dicom_datastore(project_name, location_id, my_dataset_id, my_dicom_store_id)

datastores that exist for project_id bwh-midrc-rapid-res-1655321320, location us-central1, dataset deepa: ['deepa2', 'deepa_seg_and_sr']
Created dicomStore [deepa].

Created DICOM datastore deepa
datastores that exist for project_id bwh-midrc-rapid-res-1655321320, location us-central1, dataset deepa: ['deepa', 'deepa2', 'deepa_seg_and_sr']


In [43]:
create_gcp_healthcare_dicom_datastore(project_name, location_id, my_dataset_id, my_dicom_store_id_seg_and_sr)

datastores that exist for project_id bwh-midrc-rapid-res-1655321320, location us-central1, dataset deepa: ['deepa2']
Created dicomStore [deepa_seg_and_sr].

Created DICOM datastore deepa_seg_and_sr
datastores that exist for project_id bwh-midrc-rapid-res-1655321320, location us-central1, dataset deepa: ['deepa2', 'deepa_seg_and_sr']


# Ingest raw/SEG/SR to the DICOM datastore called 'deepa'

## Ingest all raw data to the DICOM datastore

In [32]:
print(my_dataset_id) 
print(my_dicom_store_id)
print(bucket_name_midrc_data)

import time 
start_time = time.time()
!gcloud healthcare dicom-stores import gcs $my_dicom_store_id \
                                                --dataset=$my_dataset_id \
                                                --location=$location_id \
                                                --project=$project_name \
                                                --gcs-uri=gs://$bucket_name_midrc_data/** 
end_time = time.time()
print(end_time-start_time)

deepa
deepa
midrc-data-bwh
Request issued for: [deepa]
name: projects/bwh-midrc-rapid-res-1655321320/locations/us-central1/datasets/deepa/dicomStores/deepa
762.1824150085449


## Ingest only all SEG and SR to the DICOM datastore called 'deepa_seg_and_sr'

Download the BPR json files 

In [24]:
bpr_json_path = os.path.join("gs://", bucket_name, bucket_path)
bpr_folder = "/content/json"
if not os.path.isdir(bpr_folder): 
  os.mkdir(bpr_folder)
print(bpr_json_path)
print(bpr_folder)

!gsutil -m cp $bpr_json_path/*.json $bpr_folder

gs://midrc-analysis-bwh-dk/bpr-results
/content/json
Copying gs://midrc-analysis-bwh-dk/bpr-results/1.2.826.0.1.3680043.10.474.419639.105799060738901793068313281334.json...
Copying gs://midrc-analysis-bwh-dk/bpr-results/1.2.826.0.1.3680043.10.474.419639.106364025147079899289440200715.json...
Copying gs://midrc-analysis-bwh-dk/bpr-results/1.2.826.0.1.3680043.10.474.419639.259584978733948574762940092562.json...
Copying gs://midrc-analysis-bwh-dk/bpr-results/1.2.826.0.1.3680043.10.474.419639.149051607502633615235577977952.json...
Copying gs://midrc-analysis-bwh-dk/bpr-results/1.2.826.0.1.3680043.10.474.419639.189346812051260775638656981947.json...
Copying gs://midrc-analysis-bwh-dk/bpr-results/1.2.826.0.1.3680043.10.474.419639.269534881585852761235289087419.json...
Copying gs://midrc-analysis-bwh-dk/bpr-results/1.2.826.0.1.3680043.10.474.419639.198735019931123383691750806063.json...
Copying gs://midrc-analysis-bwh-dk/bpr-results/1.2.826.0.1.3680043.10.474.419639.27211765717831873215042316

Check which ones have the CHEST as a region and include that in the list of series_ids_chest_bpr

In [25]:
bpr_files = [os.path.join(bpr_folder,f) for f in os.listdir(bpr_folder)]
num_bpr_files = len(bpr_files)
print('num_bpr_files: ' + str(num_bpr_files))
series_ids_chest_bpr = [] 

for n in range(0,num_bpr_files): 
  fileObject = open(bpr_files[n], "r")
  jsonContent = fileObject.read()
  aList = json.loads(jsonContent)
  # get the list of bodyparts 
  bodypart_list = [f[0] for f in list(aList['body part examined'].items())]
  # get index of chest 
  chest_index = bodypart_list.index("chest")
  # then check if chest corresponding list is empty or not 
  chest_list = list(aList['body part examined'].items())[chest_index][1]
  # if chest_list is not empty, add SeriesInstanceUID
  if (chest_list): 
    filename = os.path.basename(bpr_files[n])
    series_ids_chest_bpr.append(os.path.splitext(filename)[0])

print('num series_ids_chest_bpr: ' + str(len(series_ids_chest_bpr)))

num_bpr_files: 379
num series_ids_chest_bpr: 369


Then get a list of the series that have the lung segmented - series_ids_lung

In [26]:
def listdir_bucket(project_name, bucket_name, dir_gs_uri):
  
  """
  Export DICOM SEG object from segmentation masks stored in NRRD files.

  Arguments:
    project_name : required - name of the GCP project.
    bucket_name  : required - name of the bucket (without gs://)
    file_gs_uri  : required - directory GS URI
  
  Returns:
    file_list : list of files in the specified GCS bucket.

  Outputs:
    This function [...]
  """

  storage_client = storage.Client(project = project_name)
  bucket = storage_client.get_bucket(bucket_name)
  
  bucket_gs_url = "gs://%s/"%(bucket_name)
  path_to_dir_relative = dir_gs_uri.split(bucket_gs_url)[-1]


  print("Getting the list of files at `%s`..."%(dir_gs_uri))

  file_list = list()

  for blob in storage_client.list_blobs(bucket_name,  prefix = path_to_dir_relative):
    fn = os.path.basename(blob.name)
    file_list.append(fn)

  return file_list

In [27]:

# exclude from processing all the patients for which a DICOM SEG object was exported already
# (stored in the specified Google Cloud Storage Bucket)
bucket_seg_folder_uri_bpr = os.path.join("gs://", bucket_name, bucket_path)
print("bucket_seg_folder_uri_bpr: " + str(bucket_seg_folder_uri_bpr))

json_bucket_list = listdir_bucket(project_name = project_name, # or gcs.listdir_bucket
                                  bucket_name = bucket_name,
                                  dir_gs_uri = bucket_seg_folder_uri_bpr)

# already_processed_id_list = [f.split("_SEG")[0] for f in json_bucket_list if f.endswith(".json")]
# already_processed_id_list = [f.split(".json")[0] for f in json_bucket_list if f.endswith(".json")]
already_processed_id_list = [f.split(".dcm")[0] for f in json_bucket_list if f.endswith(".dcm")] # or seg_ if ends with .nrrd
series_ids_lung = [f.split("dcmseg_")[1] for f in already_processed_id_list]

print('num series_ids_lung: ' + str(len(series_ids_lung)))

bucket_seg_folder_uri_bpr: gs://midrc-analysis-bwh-dk/bpr-results
Getting the list of files at `gs://midrc-analysis-bwh-dk/bpr-results`...
num series_ids_lung: 377


Intersect the two lists, and only add this list of series_ids SEG and SR to the DICOM datastore 

In [28]:
series_ids_keep = list(set(series_ids_chest_bpr) & set(series_ids_lung))
print('num series_ids_keep: ' + str(len(series_ids_keep)))

num series_ids_keep: 369


Ingest all SEG and (current) SR to the DICOM datastore from series_ids_keep

First create a text file of all of the gs urls for the dicom files we want to copy 

In [37]:
seg_and_sr_list = [] 
download_path = os.path.join("gs://", bucket_name, bucket_path_sr_and_seg)
print(download_path)

for n in range(0,len(series_ids_keep)):
  series_id = series_ids_keep[n]
  # add dicom seg name 
  filename = os.path.join("gs://", bucket_name, bucket_path, 'dcmseg_' + str(series_id) + ".dcm")
  seg_and_sr_list.append(filename)
  # add radiomics name 
  filename = os.path.join("gs://", bucket_name, bucket_path_sr, "sr_radiomics_" + str(series_id) + ".dcm")
  seg_and_sr_list.append(filename)
  # add regions name 
  filename = os.path.join("gs://", bucket_name, bucket_path_sr, "sr_regions_" + str(series_id) + ".dcm")
  seg_and_sr_list.append(filename)
  # add landmarks name 
  filename = os.path.join("gs://", bucket_name, bucket_path_sr, "sr_landmarks_" + str(series_id) + ".dcm")
  seg_and_sr_list.append(filename)
# create dataframe 
series_df = pd.DataFrame() 
series_df["gcs_url"] = seg_and_sr_list 
# gcsurl_temp = "cp " + series_df["gcs_url"] + " " + download_path 
gcsurl_temp = series_df["gcs_url"]
gs_file_path = "gcs_paths.txt"
gcsurl_temp.to_csv(gs_file_path, header = False, index = False)


gs://midrc-analysis-bwh-dk/seg_and_sr


In [35]:
series_df

,gcs_url
0,gs://midrc-analysis-bwh-dk/bpr-results/dcmseg_...
1,gs://midrc-analysis-bwh-dk/structured_reports/...
2,gs://midrc-analysis-bwh-dk/structured_reports/...
3,gs://midrc-analysis-bwh-dk/structured_reports/...
4,gs://midrc-analysis-bwh-dk/bpr-results/dcmseg_...
...,...
1471,gs://midrc-analysis-bwh-dk/structured_reports/...
1472,gs://midrc-analysis-bwh-dk/bpr-results/dcmseg_...
1473,gs://midrc-analysis-bwh-dk/structured_reports/...
1474,gs://midrc-analysis-bwh-dk/structured_reports/...


Then copy these SEG and SR files into a temporary bucket 

In [38]:
!cat "/content/gcs_paths.txt" | gsutil -m cp -I $download_path
# $ cat gcs_paths.txt | gsutil -m cp -I .

Copying gs://midrc-analysis-bwh-dk/bpr-results/dcmseg_2.16.840.1.114274.1818.550364590190018697815472723977627139207.dcm [Content-Type=application/dicom]...
Copying gs://midrc-analysis-bwh-dk/structured_reports/sr_radiomics_2.16.840.1.114274.1818.550364590190018697815472723977627139207.dcm [Content-Type=application/dicom]...
Copying gs://midrc-analysis-bwh-dk/structured_reports/sr_regions_2.16.840.1.114274.1818.550364590190018697815472723977627139207.dcm [Content-Type=application/dicom]...
Copying gs://midrc-analysis-bwh-dk/structured_reports/sr_landmarks_2.16.840.1.114274.1818.550364590190018697815472723977627139207.dcm [Content-Type=application/dicom]...
Copying gs://midrc-analysis-bwh-dk/bpr-results/dcmseg_2.16.840.1.114274.1818.53053387282722816036336839742290626991.dcm [Content-Type=application/dicom]...
Copying gs://midrc-analysis-bwh-dk/structured_reports/sr_radiomics_2.16.840.1.114274.1818.53053387282722816036336839742290626991.dcm [Content-Type=application/dicom]...
Copying gs

Copy the files that we will ingest to the DICOM datastore to a bucket first -- too slow. 

In [ ]:
# for n in range(0,len(series_ids_keep)): 
# # for n in range(0,1): 
#   series_id = series_ids_keep[n]
#   print(n)
#   # copy SEG 
#   input_path = os.path.join("gs://", bucket_name, bucket_path, "dcmseg_" + str(series_id) + ".dcm")
#   output_dir = os.path.join("gs://", bucket_name, bucket_path_sr_and_seg)
#   !gsutil -m cp $input_path $output_dir
#   # copy the landmarks SR 
#   input_path = os.path.join("gs://", bucket_name, bucket_path_sr, "sr_landmarks_" + str(series_id) + ".dcm")
#   output_dir = os.path.join("gs://", bucket_name, bucket_path_sr_and_seg)
#   !gsutil -m cp $input_path $output_dir
#   # copy the regions SR 
#   input_path = os.path.join("gs://", bucket_name, bucket_path_sr, "sr_regions_" + str(series_id) + ".dcm")
#   output_dir = os.path.join("gs://", bucket_name, bucket_path_sr_and_seg)
#   !gsutil -m cp $input_path $output_dir
#   # copy the features SR 
#   input_path = os.path.join("gs://", bucket_name, bucket_path_sr, "sr_radiomics_" + str(series_id) + ".dcm")
#   output_dir = os.path.join("gs://", bucket_name, bucket_path_sr_and_seg)
#   !gsutil -m cp $input_path $output_dir

Then ingest into DICOM datastore

In [45]:
# Ingest SEG and SR from single bucket

my_bucket_path = os.path.join(bucket_name, bucket_path_sr_and_seg)
print(my_bucket_path)

!gcloud healthcare dicom-stores import gcs $my_dicom_store_id_seg_and_sr \
                                                --dataset=$my_dataset_id \
                                                --location=$location_id \
                                                --project=$project_name \
                                                --gcs-uri=gs://$my_bucket_path/**.dcm
# #for n in range(0,len(series_ids_keep)):
# for n in range(0,1):
#   filename = os.path.join("gs://", my_bucket_path, "dcmseg_" + str(series_ids_keep[n]) + ".dcm")
#   print(str(n) + " : " + str(filename))
#   !gcloud healthcare dicom-stores import gcs $my_dicom_store_id_seg_and_sr \
#                                                 --dataset=$my_dataset_id \
#                                                 --location=$location_id \
#                                                 --project=$project_name \
#                                                 --gcs-uri=$filename

midrc-analysis-bwh-dk/seg_and_sr
Request issued for: [deepa_seg_and_sr]
name: projects/bwh-midrc-rapid-res-1655321320/locations/us-central1/datasets/deepa/dicomStores/deepa_seg_and_sr


In [33]:
# # Ingest 

# my_bucket_path = os.path.join(bucket_name, bucket_path_sr)
# print(my_bucket_path)

# !gcloud healthcare dicom-stores import gcs $my_dicom_store_id_seg_and_sr \
#                                                 --dataset=$my_dataset_id \
#                                                 --location=$location_id \
#                                                 --project=$project_name \
#                                                 --gcs-uri=gs://$my_bucket_path**.dcm

midrc-analysis-bwh-dk/structured_reports/
Request issued for: [deepa_seg_and_sr]
name: projects/bwh-midrc-rapid-res-1655321320/locations/us-central1/datasets/deepa/dicomStores/deepa_seg_and_sr


## Now copy from the bucket with seg and sr to the DICOM datastore called deepa

In [33]:
bucket_path_sr_and_seg_full = os.path.join(bucket_name,bucket_path_sr_and_seg)
print(project_name)
print(location_id)
print(my_dataset_id)
print(my_dicom_store_id)
print(bucket_path_sr_and_seg_full)

!gcloud healthcare dicom-stores import gcs $my_dicom_store_id \
                                                --dataset=$my_dataset_id \
                                                --location=$location_id \
                                                --project=$project_name \
                                                --gcs-uri=gs://$bucket_path_sr_and_seg_full/**.dcm

bwh-midrc-rapid-res-1655321320
us-central1
deepa
deepa
midrc-analysis-bwh-dk/seg_and_sr
Request issued for: [deepa]
name: projects/bwh-midrc-rapid-res-1655321320/locations/us-central1/datasets/deepa/dicomStores/deepa


# IGNORE - Temporary - Export from midrc-dicom dicom datastore into bucket, and then reingest into my own dicom datastore. 

Create a new bucket in the same location as the dicom datastore (us-central1)

In [25]:
!gsutil mb -l "us-central1" -p $project_name "gs://midrc-raw-data-dk"

Creating gs://midrc-raw-data-dk/...


In [27]:
print(dataset_id)
print(dicom_store_id)

!gcloud healthcare dicom-stores export gcs $dicom_store_id \
  --project=$project_name --dataset=$dataset_id --location="us-central1" --gcs-uri-prefix=gs://midrc-raw-data-dk/


midrc
midrc-dicom
ERROR: (gcloud.healthcare.dicom-stores.export.gcs) PERMISSION_DENIED: Cloud Healthcare Service Agent doesn't have all required permissions [storage.objects.create] on resource "gs://midrc-raw-data-dk" (or the resource may not exist)


In [22]:
# print(dataset_id)
# print(dicom_store_id)
# print(raw_data_bucket)

# !gcloud healthcare dicom-stores export gcs $dicom_store_id \
#   --project=$project_name --dataset=$dataset_id --location=$location_id --gcs-uri-prefix=gs://midrc-analysis-bwh-dk/raw_data/


  # --location=$location_id \

    # --gcs-uri-prefix=gs://$raw_data_bucket/**.dcm


midrc
midrc-dicom
midrc-analysis-bwh-dk/raw_data
ERROR: (gcloud.healthcare.dicom-stores.export.gcs) PERMISSION_DENIED: Cloud Healthcare Service Agent doesn't have all required permissions [storage.objects.create] on resource "gs://midrc-analysis-bwh-dk" (or the resource may not exist)


Until I find where the bucket that has the raw data is stored.

# IGNORE - WIP Ingest the CT to the DICOM datastore 

Ingest to DICOM datastore

In [39]:
def download_series(study_instance_uid, series_instance_uid, sop_instance_uids, dest_dir):
  import pydicom
  token = !gcloud auth print-access-token
  token = token[0]

  PROJECT_ID="bwh-midrc-rapid-res-1655321320"
  REGION="us-central1"

  # DATASET_ID="midrc"
  # DICOM_STORE_ID="midrc-dicom"
  
  my_project = "bwh-midrc-rapid-res-1655321320"
  location = "us-central1"
  dataset_id = "midrc"
  dicom_store_id = "midrc-dicom"

  # url = f"https://healthcare.googleapis.com/v1/projects/{my_project}/locations/{location}/datasets/{dataset_id}/dicomStores/{dicom_store_id}/dicomWeb"
  # headers = {
  #     "Authorization" : "Bearer %s" % token
  # }

  # import dicomweb_client

  # client = dicomweb_client.api.DICOMwebClient(url, headers=headers)

  # idx=0
  # for sop_instance_uid in sop_instance_uids:
  #   retrievedInstance = client.retrieve_instance(
  #               study_instance_uid=study_instance_uid,
  #               series_instance_uid=series_instance_uid,
  #               sop_instance_uid=sop_instance_uid)
  #   pydicom.filewriter.write_file(f"{dest_dir}/file{idx}.dcm", retrievedInstance)
  #   idx+=1

  # instances = client.retrieve_series(
  #                                     study_instance_uid=study_instance_uid,
  #                                     series_instance_uid=series_instance_uid)
  # idx=0
  # for retrievedInstance in instances: 
  #   pydicom.filewriter.write_file(f"{dest_dir}/file{idx}.dcm", retrievedInstance)
  #   idx+=1

  url = os.path.join('https://healthcare.googleapis.com/v1', 
                     'projects', my_project, 
                     'locations', 'us-central1', 
                     'datasets', dataset_id, 
                     'dicomStores', dicom_store_id, 
                     'dicomWeb/')
  url_study_and_series = os.path.join('studies', 
                                      study_instance_uid,
                                      'series', 
                                      series_instance_uid)
  print (url)
  print(url_study_and_series)

  !dcmweb -m $url retrieve $url_study_and_series $dest_dir
  
  input_dir = os.path.join(dest_dir, study_instance_uid, series_instance_uid)
  !mv $input_dir/* $dest_dir

  # !dcmweb -m \
  # https://healthcare.googleapis.com/v1/projects/bwh-midrc-rapid-res-1655321320/locations/us-central1/datasets/midrc/dicomStores/midrc-dicom/dicomWeb/ \
  # retrieve studies/2.16.840.1.114274.1818.562777179262603788569215634484746132/series/2.16.840.1.114274.1818.46652472987550009305366336206019799966	\
  # "/content/dcmweb"
  

In [40]:
seriesuid = "1.3.6.1.4.1.5962.99.1.1247991691.71736179.1616155726490.22602.0"

df = ct_limited_open_a1_r1

dest_dir = "/content/raw"
if not os.path.isdir("/content/raw"):
  os.mkdir("/content/raw")

studyuids = list(set(df[df['SeriesInstanceUID']==seriesuid]['StudyInstanceUID'].tolist()))
if len(studyuids)!=1:
  print('error - not one UID for this series', studyuids)
  print(studyuids)
  1/0
studyuid = studyuids[0]
sops = df[ (df['StudyInstanceUID']==studyuid) & (df['SeriesInstanceUID']==seriesuid) ]['SOPInstanceUID'].tolist()


In [41]:
# Download CT files using dcmweb 
dest_dir = os.path.join("/content/raw", studyuid, seriesuid)
if not os.path.isdir(dest_dir):
  os.makedirs(dest_dir)
print(dest_dir)

download_series(studyuid, seriesuid, sops, dest_dir)


/content/raw/1.3.6.1.4.1.5962.99.1.1247991691.71736179.1616155726490.22155.0/1.3.6.1.4.1.5962.99.1.1247991691.71736179.1616155726490.22602.0
https://healthcare.googleapis.com/v1/projects/bwh-midrc-rapid-res-1655321320/locations/us-central1/datasets/midrc/dicomStores/midrc-dicom/dicomWeb/
studies/1.3.6.1.4.1.5962.99.1.1247991691.71736179.1616155726490.22155.0/series/1.3.6.1.4.1.5962.99.1.1247991691.71736179.1616155726490.22602.0
2023-01-21 00:58:12,527 -- No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
2023-01-21 00:58:12,793 -- Saving files into /content/raw/1.3.6.1.4.1.5962.99.1.1247991691.71736179.1616155726490.22155.0/1.3.6.1.4.1.5962.99.1.1247991691.71736179.1616155726490.22602.0
2023-01-21 00:58:14,384 -- Transferred 16M in 33 files     [
2023-01-21 00:58:15,385 -- Transferred 36M in 72 files     [
2023-01-21 00:58:16,387 -- Transferred 56M in 113 files     [
2023-01-21 00:58:17,389 -- Tran

In [42]:
print(my_dataset_id)
print(my_dicom_store_id)

deepa
deepa


In [43]:
# Store the DICOM instances using dcmweb
url = os.path.join('https://healthcare.googleapis.com/v1', 
                    'projects', project_name, 
                    'locations', location_id, 
                    'datasets', my_dataset_id, 
                    'dicomStores', my_dicom_store_id, 
                    'dicomWeb/')
!dcmweb -m $url store $dest_dir/**.dcm 


# dcmweb -m \
#   https://healthcare.googleapis.com/v1/projects/PROJECT_ID/locations/LOCATION/datasets/DATASET_ID/dicomStores/DICOM_STORE_ID/dicomWeb \
#   store "./**.dcm"

2023-01-21 00:59:08,236 -- No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
2023-01-21 00:59:09,698 -- /content/raw/1.3.6.1.4.1.5962.99.1.1247991691.71736179.1616155726490.22155.0/1.3.6.1.4.1.5962.99.1.1247991691.71736179.1616155726490.22602.0/1.3.6.1.4.1.5962.99.1.1247991691.71736179.1616155726490.22601.0.dcm uploaded as https://healthcare.googleapis.com/v1/projects/bwh-midrc-rapid-res-1655321320/locations/us-central1/datasets/deepa/dicomStores/deepa/dicomWeb/studies/1.3.6.1.4.1.5962.99.1.1247991691.71736179.1616155726490.22155.0/series/1.3.6.1.4.1.5962.99.1.1247991691.71736179.1616155726490.22602.0/instances/1.3.6.1.4.1.5962.99.1.1247991691.71736179.1616155726490.22601.0
2023-01-21 00:59:09,698 -- /content/raw/1.3.6.1.4.1.5962.99.1.1247991691.71736179.1616155726490.22155.0/1.3.6.1.4.1.5962.99.1.1247991691.71736179.1616155726490.22602.0/1.3.6.1.4.1.5962.99.1.1247991691.71736179.1616155726490.22605.

In [56]:
# add appropriate DICOM SEG for now 

# copy from bucket to DICOM datastore  

my_bucket_path = os.path.join(bucket_name, bucket_path)
dicomseg_fn = "dcmseg_" + seriesuid + ".dcm"
print(my_bucket_path)
print(dicomseg_fn)

!gcloud healthcare dicom-stores import gcs $my_dicom_store_id \
                                                --dataset=$my_dataset_id \
                                                --location=$location_id \
                                                --project=$project_name \
                                                --gcs-uri=gs://$my_bucket_path/$dicomseg_fn

midrc-analysis-bwh-dk/bpr-results
dcmseg_1.3.6.1.4.1.5962.99.1.1247991691.71736179.1616155726490.22602.0.dcm
Request issued for: [deepa]
name: projects/bwh-midrc-rapid-res-1655321320/locations/us-central1/datasets/deepa/dicomStores/deepa


In [60]:
# Create url 
url = os.path.join("https://idc-tester-1.web.app/projects/bwh-midrc-rapid-res-1655321320/locations/us-central1/datasets/", 
                   my_dataset_id, 
                   "dicomStores", 
                   my_dicom_store_id, 
                   "study", 
                   studyuid)
print(url)

# (instead of storing in bucket and then importing into DICOM datastore)

# https://idc-tester-1.web.app/projects/bwh-midrc-rapid-res-1655321320/locations/us-central1/datasets/afedorov-hc-dataset/dicomStores/dicom/study/2.16.840.1.114274.1818.48737721983267329291044630832848791991

https://idc-tester-1.web.app/projects/bwh-midrc-rapid-res-1655321320/locations/us-central1/datasets/deepa/dicomStores/deepa/study/1.3.6.1.4.1.5962.99.1.1247991691.71736179.1616155726490.22155.0
